# 02. Learning LangGraph - Chat Executor

modified from https://github.com/langchain-ai/langgraph/blob/main/examples/chat_agent_executor_with_function_calling/base.ipynb

In [1]:
import os
from dotenv import load_dotenv

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv('LANGSMITH_API_KEY')
os.environ["LANGCHAIN_PROJECT"] = "LangGraph_01"

## The model

In [3]:
from langchain import hub
from langchain.agents import create_structured_chat_agent
from tools.initialize_cerebras import init_cerebras

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/structured-chat-agent")

# Choose the LLM that will drive the agent
client, llm = init_cerebras()


## Tools

In [4]:
from langchain.tools import BaseTool, StructuredTool, Tool, tool
import random

@tool("lower_case", return_direct=True)
def to_lower_case(input:str) -> str:
  """Returns the input as all lower case."""
  return input.lower()

@tool("random_number", return_direct=True)
def random_number_maker(input:str) -> str:
    """Returns a random number between 0-100. input the word 'random'"""
    return random.randint(0, 100)

tools = [to_lower_case,random_number_maker]

In [14]:
from langgraph.prebuilt.tool_executor import ToolExecutor

tool_executor = ToolExecutor(tools)

C:\Users\ncvn\AppData\Local\Temp\ipykernel_12644\3389706677.py:3: LangGraphDeprecationWarning: ToolExecutor is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  tool_executor = ToolExecutor(tools)


In [13]:
# from langchain.tools.render import render_text_description

# functions = [render_text_description(t) for t in tools]
# model = llm.bind_functions(functions)

## AgentState

In [15]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

## Nodes

In [20]:
from langchain_core.agents import AgentFinish
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage

# Define the function that determines whether to continue or not
def should_continue(state):
    messages = state['messages']
    last_message = messages[-1]
    # If there is no function call, then we finish
    if "function_call" not in last_message.additional_kwargs:
        return "end"
    # Otherwise if there is, we continue
    else:
        return "continue"

# Define the function that calls the model
def call_model(state):
    messages = state['messages']
    response = llm.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

# Define the function to execute tools
def call_tool(state):
    messages = state['messages']
    # Based on the continue condition
    # we know the last message involves a function call
    last_message = messages[-1]
    # We construct an ToolInvocation from the function_call
    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=json.loads(last_message.additional_kwargs["function_call"]["arguments"]),
    )
    print(f"The agent action is {action}")
    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)
    print(f"The tool result is: {response}")
    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(content=str(response), name=action.tool)
    # We return a list, because this will get added to the existing list
    return {"messages": [function_message]}

## Graph

In [21]:
from langgraph.graph import StateGraph, END
# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)

# Set the entrypoint as `agent` where we start
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Otherwise we finish.
        "end": END
    }
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge('action', 'agent')

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

## Run it

In [22]:
from langchain_core.messages import HumanMessage, SystemMessage
# inputs = {"input": "give me a random number and then write in words and make it lower case", "chat_history": []}

system_message = SystemMessage(content="you are a helpful assistant")
user_01 = HumanMessage(content="give me a random number and then write in words and make it lower case")
# user_01 = HumanMessage(content="plear write 'Merlion' in lower case")
# user_01 = HumanMessage(content="what is a Merlion?")

inputs = {"messages": [system_message,user_01]}

app.invoke(inputs)

{'messages': [SystemMessage(content='you are a helpful assistant', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='give me a random number and then write in words and make it lower case', additional_kwargs={}, response_metadata={}),
  AIMessage(content='the random number is: 854\n\neight hundred fifty-four', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'llama3.1-70b', 'system_fingerprint': 'fp_fe26f95222'}, id='run-83e27441-9e57-4be5-8ed5-4d7ffbbdf21c-0', usage_metadata={'input_tokens': 55, 'output_tokens': 13, 'total_tokens': 68, 'input_token_details': {}, 'output_token_details': {}})]}

In [23]:
from langchain_core.messages import HumanMessage, SystemMessage
# inputs = {"input": "give me a random number and then write in words and make it lower case", "chat_history": []}

system_message = SystemMessage(content="you are a helpful assistant")
# user_01 = HumanMessage(content="give me a random number and then write in words and make it lower case")
user_01 = HumanMessage(content="plear write 'Merlion' in lower case")
# user_01 = HumanMessage(content="what is a Merlion?")

inputs = {"messages": [system_message,user_01]}

app.invoke(inputs)

{'messages': [SystemMessage(content='you are a helpful assistant', additional_kwargs={}, response_metadata={}),
  HumanMessage(content="plear write 'Merlion' in lower case", additional_kwargs={}, response_metadata={}),
  AIMessage(content='merlion', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'llama3.1-70b', 'system_fingerprint': 'fp_fe26f95222'}, id='run-c8d610c8-9e59-4b78-a473-e5e7ab65918c-0', usage_metadata={'input_tokens': 50, 'output_tokens': 3, 'total_tokens': 53, 'input_token_details': {}, 'output_token_details': {}})]}

In [24]:
from langchain_core.messages import HumanMessage, SystemMessage
# inputs = {"input": "give me a random number and then write in words and make it lower case", "chat_history": []}

system_message = SystemMessage(content="you are a helpful assistant")
# user_01 = HumanMessage(content="give me a random number and then write in words and make it lower case")
# user_01 = HumanMessage(content="plear write 'Merlion' in lower case")
user_01 = HumanMessage(content="what is a Merlion?")

inputs = {"messages": [system_message,user_01]}

app.invoke(inputs)

{'messages': [SystemMessage(content='you are a helpful assistant', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='what is a Merlion?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="The Merlion is a mythical creature and a national symbol of Singapore. It has the body of a fish (typically a lionfish) and the head of a lion. The name 'Merlion' is a combination of the words 'mermaid' (or 'mer' as in 'fish') and 'lion.'\n\nAccording to legend, the Merlion was created by Prince Sang Nila Utama, a prince from the Malay kingdom of Palembang. In the 14th century, the prince sailed to the island of Temasek (now Singapore) and saw a majestic lion on the island. He decided to name the island 'Singapura,' which means 'lion city' in Sanskrit.\n\nHowever, the Merlion statue that is famous today was actually designed by Fraser Brunner, a British artist, in 1964, as part of a campaign to promote Singapore as a tourist destination. The most famous Merlion statu